In [1]:
import splinepy
import vedo
import numpy as np
from sdf_microstructure import (
    create_microstructure_from_experiment,
    export_mesh,
    tetrahedralize_surface,
)
import os
os.chdir("/home/michael.kofler/DeepSDF")
from analysis.problems.CantileverBeam import CantileverBeam as le

vedo.settings.default_backend = "k3d"
resolution=40

In [ ]:
rect = splinepy.helpme.create.box(2, 1, 1)
rect.control_points[1][2] = 0.1
rect.control_points[3][2] = 0.1
volumes = rect.extract.volumes(resolution=resolution)
rect_dist = splinepy.helpme.create.box(2, 1, 1)
print(rect_dist.control_points)
rect_dist.control_points[1][2] = 0.001
rect_dist.control_points[3][2] = 0.001
volumes = rect.extract.volumes(resolution=resolution)
volumes_dist = rect_dist.extract.volumes(resolution=resolution)

[[0. 0. 0.]
 [2. 0. 0.]
 [0. 1. 0.]
 [2. 1. 0.]
 [0. 0. 1.]
 [2. 0. 1.]
 [0. 1. 1.]
 [2. 1. 1.]]


In [3]:
export_mesh(volumes, "test_mesh.mesh", show_mesh=False)
export_mesh(volumes_dist, "test_mesh_dist.mesh", show_mesh=True)

Exporting mesh with 59319 elements, 64000 vertices, 1521 boundaries with marker 1, 1677 boundaries with marker 2, and 5928 boundaries with marker 3.


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

Exporting mesh with 59319 elements, 64000 vertices, 1521 boundaries with marker 1, 1677 boundaries with marker 2, and 5928 boundaries with marker 3.


In [4]:
# dVertices of stretched - base
dVertices = volumes_dist.vertices - volumes.vertices
dVertices = np.dstack([dVertices])

In [5]:
le_problem = le("simulation/temp_sim")
le_problem.read_mesh("test_mesh.mesh")
# before solve, we should add a problem setup and set material properties
le_problem.set_up()
vol, der_vol = le_problem.compute_volume(dTheta=dVertices)
print(f"Vol: {vol:.5g}, dVol: {der_vol}")
le_problem.solve()
compliance, der_compliance = le_problem.compute_compliance(dTheta=dVertices)
print(f"Compliance: {compliance:.5g}, dCompliance: {der_compliance}")
le_problem.show_solution(output=["u_vec","strain_energy_density"])

Vol: 1.9, dVol: [0.0990029]
Compliance: 2.2414e-05, dCompliance: [-1.63551659e-06]


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [6]:
le_problem = le("simulation/temp_sim")
le_problem.read_mesh("test_mesh_dist.mesh")
# before solve, we should add a problem setup and set material properties
le_problem.set_up()
vol, der_vol = le_problem.compute_volume(dTheta=dVertices)
print(f"Vol: {vol:.5g}, dVol: {der_vol}")
le_problem.solve()
compliance_new, _ = le_problem.compute_compliance(dTheta=dVertices)
print(f"Compliance: {compliance_new:.5g}")
print(f"Delta: {compliance_new-compliance}, expected: {der_compliance}")
le_problem.show_solution(output=["u_vec","strain_energy_density"])

Vol: 1.999, dVol: [0.09900064]
Compliance: 2.5286e-05
Delta: 2.8722439812296683e-06, expected: [-1.63551659e-06]


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [7]:
# le_problem = le.LinearElasticityProblem()
# le_problem.read_mesh("test_mesh_stretched.mesh")
# # before solve, we should add a problem setup and set material properties
# le_problem.set_up(ref_levels=0)
# vol, _ = le_problem.compute_volume()
# print(f"Volume of deformed mesh {vol:.5g}")
# le_problem.solve()
# compliance, der_compliance = le_problem.compute_compliance()
# print(f"Compliance of deformed mesh: {compliance:.5g}")
# le_problem.show_solution(output=["u_vec","strain_energy_density"])